In [10]:
#!pip install -q gradio langchain sentence-transformers faiss-cpu transformers

import gradio as gr
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from transformers import pipeline

corpus = [
    "I love this product! It has changed my life.",
    "The service was terrible. I am very disappointed.",
    "Amazing experience, will definitely come back again!",
    "Not worth the money. The quality was poor.",
    "I'm neutral about the update. It didn’t do much."
]

def setup_vector_store(corpus):
    docs = [Document(page_content=t) for t in corpus]
    splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
    split_docs = splitter.split_documents(docs)
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(split_docs, embeddings)
    return vectorstore.as_retriever(search_kwargs={"k": 2})

retriever = setup_vector_store(corpus)
classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

def sentiment_rag_interface(query):
    results = retriever.get_relevant_documents(query)
    response = ""
    for i, doc in enumerate(results):
        sentiment = classifier(doc.page_content)[0]
        response += f"Match {i+1}:\n{doc.page_content}\n→ Sentiment: {sentiment['label']} (Confidence: {sentiment['score']:.2f})\n\n"
    return response.strip()

with gr.Blocks(title="💬 Sentiment RAG Analyzer") as demo:
    gr.Markdown("## Ask questions to analyze sentiment from corpus using RAG + Transformers")
    with gr.Row():
        query_input = gr.Textbox(label="Your Question", placeholder="e.g. What do users feel about quality?")
    output_box = gr.Textbox(label="RAG Sentiment Output", lines=8)
    btn = gr.Button("Analyze")

    btn.click(sentiment_rag_interface, inputs=query_input, outputs=output_box)

demo.launch(debug=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.1 MB/s eta 0:00:00


Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e79e344203229b7a22.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e79e344203229b7a22.gradio.live
